## Tutorial on how to use the Mixing Matrix

In [18]:
import numpy as np
from micmac import *

#### Set required parameters
(In actual applications they'll come from the instrument / input data)

In [2]:
# nside of input freq maps
NSIDE = 64
# choose frequencies
freqs = np.array([40, 50, 60, 100, 140])
# choose number of components
ncomp = 3
# choose frequencies at which we have one fgs and the others zero
# in MixingMat columns first synch, then dust
pos_special_freqs = np.array([0, -1])
# number of parameters in the MixingMat
n_betas = (ncomp-1)*(len(freqs)-ncomp+1)

In [27]:
root_tree = tree_spv_config("", 6, 2, print_tree=True)

mixing_matrix_test = MixingMatrix(frequency_array=np.array([40, 50, 60, 100, 140,220]), 
                                  n_components=ncomp, 
                                  params=np.arange(8),
                                  pos_special_freqs=pos_special_freqs, 
                                  spv_nodes_b=get_nodes_b(root_tree), 
                                  nside=NSIDE)

No spatial variability case

>>> Tree of spv config as passed by the User:
root
  nside_spv
    default: [0]
    f0
      default: None
      b0
        default: None
      b1
        default: None
      b2
        default: None
    f1
      default: None
      b0
        default: None
      b1
        default: None
      b2
        default: None

>>> Tree of spv config after filling the missing values:
root
  nside_spv
    default: [0]
    f0
      default: [0]
      b0
        default: [0]
      b1
        default: [0]
      b2
        default: [0]
    f1
      default: [0]
      b0
        default: [0]
      b1
        default: [0]
      b2
        default: [0]


TypeError: Exception() takes no keyword arguments

#### Get spv tree

In [3]:
yaml_file_path = 'params_spv_tutorial.yaml'
root_tree = tree_spv_config(yaml_file_path, n_betas, ncomp-1, print_tree=True)
nodes_b = get_nodes_b(root_tree)

<_io.TextIOWrapper name='params_spv_tutorial.yaml' mode='r' encoding='UTF-8'>
count_b: 6
n_betas:  6

>>> Tree of spv config as passed by the User:
root
  nside_spv
    default: [4]
    f1
      default: None
      b1
        default: [32]
      b2
        default: [8]
      b3
        default: [4]
    f2
      default: [4]
      b1
        default: [32]
      b2
        default: [8]
      b3
        default: None

>>> Tree of spv config after filling the missing values:
root
  nside_spv
    default: [4]
    f1
      default: [4]
      b1
        default: [32]
      b2
        default: [8]
      b3
        default: [4]
    f2
      default: [4]
      b1
        default: [32]
      b2
        default: [8]
      b3
        default: [4]


#### Initialize parameters of the Mixing Matrix
These are the true values (done with values coming from SEDs of a MBB and a PL).
Alternativly one can skip this step and set customized values.

In [4]:
initB = InitMixingMatrix(freqs, ncomp, pos_special_freqs, nodes_b)
init_params = initB.init_params()
init_params
for i in init_params:
    print(i)

0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001699259
0.5204473001

#### Build the Mixing Matrix

In [5]:
B = MixingMatrix(freqs, ncomp, nodes_b, NSIDE, init_params, pos_special_freqs)
B_cmb = B.get_B_cmb()
B_fgs = B.get_B_fgs()
B_all = B.get_B()

In [6]:
B.params

array([0.5204473 , 0.5204473 , 0.5204473 , ..., 0.49067474, 0.49067474,
       0.49067474])

In [7]:
B.get_params_long()

array([[[0.5204473 , 0.5204473 , 0.5204473 , ..., 0.5204473 ,
         0.5204473 , 0.5204473 ],
        [0.09568486, 0.09568486, 0.09568486, ..., 0.09568486,
         0.09568486, 0.09568486]],

       [[0.30561537, 0.30561537, 0.30561537, ..., 0.30561537,
         0.30561537, 0.30561537],
        [0.17041835, 0.17041835, 0.17041835, ..., 0.17041835,
         0.17041835, 0.17041835]],

       [[0.06122007, 0.06122007, 0.06122007, ..., 0.06122007,
         0.06122007, 0.06122007],
        [0.49067474, 0.49067474, 0.49067474, ..., 0.49067474,
         0.49067474, 0.49067474]]])

In [8]:
B_test = B.get_B_fgs(jax_use=False)

In [9]:
B_test[:,:,2]

array([[1.        , 0.        ],
       [0.5204473 , 0.09568486],
       [0.30561537, 0.17041835],
       [0.06122007, 0.49067474],
       [0.        , 1.        ]])

In [10]:
print(B_all[:,:,0])

[[1.         1.         0.        ]
 [1.         0.5204473  0.09568486]
 [1.         0.30561537 0.17041835]
 [1.         0.06122007 0.49067474]
 [1.         0.         1.        ]]


In [11]:
print(B_all)

[[[1.         1.         1.         ... 1.         1.         1.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.5204473  0.5204473  0.5204473  ... 0.5204473  0.5204473  0.5204473 ]
  [0.09568486 0.09568486 0.09568486 ... 0.09568486 0.09568486 0.09568486]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.30561537 0.30561537 0.30561537 ... 0.30561537 0.30561537 0.30561537]
  [0.17041835 0.17041835 0.17041835 ... 0.17041835 0.17041835 0.17041835]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.06122007 0.06122007 0.06122007 ... 0.06122007 0.06122007 0.06122007]
  [0.49067474 0.49067474 0.49067474 ... 0.49067474 0.49067474 0.49067474]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.         0.         0.   

#### Update parameters of the Mixing Matrix

In [12]:
new_params = B.params+1
B.update_params(new_params)
B.get_params_long()

array([[[1.5204473 , 1.5204473 , 1.5204473 , ..., 1.5204473 ,
         1.5204473 , 1.5204473 ],
        [1.09568486, 1.09568486, 1.09568486, ..., 1.09568486,
         1.09568486, 1.09568486]],

       [[1.30561537, 1.30561537, 1.30561537, ..., 1.30561537,
         1.30561537, 1.30561537],
        [1.17041835, 1.17041835, 1.17041835, ..., 1.17041835,
         1.17041835, 1.17041835]],

       [[1.06122007, 1.06122007, 1.06122007, ..., 1.06122007,
         1.06122007, 1.06122007],
        [1.49067474, 1.49067474, 1.49067474, ..., 1.49067474,
         1.49067474, 1.49067474]]])

In [13]:
B_all_new = B.get_B()
B_all_new

array([[[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.5204473 , 1.5204473 , 1.5204473 , ..., 1.5204473 ,
         1.5204473 , 1.5204473 ],
        [1.09568486, 1.09568486, 1.09568486, ..., 1.09568486,
         1.09568486, 1.09568486]],

       [[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.30561537, 1.30561537, 1.30561537, ..., 1.30561537,
         1.30561537, 1.30561537],
        [1.17041835, 1.17041835, 1.17041835, ..., 1.17041835,
         1.17041835, 1.17041835]],

       [[1.        , 1.        , 1.        , ..., 1.        ,
         1.        , 1.        ],
        [1.06122007, 1.06122007, 1